# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Load libraries

Make sure you have the latest version of GraphLab Create.

In [1]:
from __future__ import division
#import graphlab
import math
import string
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
#import sframe
#products = sframe.SFrame('amazon_baby.gl/')


# Data preparation 

## 1. Load Amazon dataset

We will use a dataset consisting of baby product reviews on Amazon.com.
* Load the dataset consisting of baby product reviews on Amazon.com. 
* Store the data in a data frame products. 

In [2]:
products = pd.read_csv('amazon_baby.csv')

Now, let us see a preview of what the dataset looks like.

In [3]:
products.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Build the word count vector for each review


Let us explore a specific example of a baby product.


In [4]:
products.loc[269]

name      The First Years Massaging Action Teether
review                    A favorite in our house!
rating                                           5
Name: 269, dtype: object

## 2. Perform text cleaning

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.



### Sklearn instruction
* We start by removing punctuation, so that words "cake." and "cake!" are counted as the same word.

Write a function remove_punctuation that strips punctuation from a line of text
Apply this function to every element in the review column of products, and save the result to a new column review_clean
```python
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

products['review_clean'] = products['review'].apply(remove_punctuation)
```

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.


**IMPORTANT**. Make sure to fill n/a values in the **review** column with empty strings (if applicable). The n/a values indicate empty reviews. For instance, Pandas's the fillna() method lets you replace all N/A's in the **review** columns as follows:


In [5]:
products = products.fillna({'review':''})  # fill in N/A's in the review column

In [8]:
def remove_punctuation(text):
    import string
    try:
        return text.translate(None, string.punctuation) 
    except:
        return ""
    
products['review_clean']  = products['review'].apply(remove_punctuation)

#products['word_count'] = graphlab.text_analytics.count_words(review_without_punctuation)
#Convert the content of string/dict/list type SArrays to a dictionary of (word, count) pairs. 
#The strings are first tokenized into words according to the specified to_lower and delimiters options. Then, word counts are accumulated. 

In [9]:
products = products.fillna({'review_clean':''})  # fill in N/A's in the review column

## 3. Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [10]:
print(len(products))
products = products[products['rating'] != 3]
len(products)
products = products.reset_index()

183531


Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [11]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products.sample(5)

,index,name,review,rating,review_clean,sentiment
82623,90959,"MAM Bite and Brush Teether, 3 Months, Colors M...","My search criteria on Amazon.com was ""made in ...",1,My search criteria on Amazoncom was made in th...,-1
45782,50350,Baby Einstein Discover &amp; Play Color Block,"My 4 month old likes to grip objects, and ther...",2,My 4 month old likes to grip objects and there...,-1
133779,147173,BRICA Corner Bath Basket Toy Organizer,This sticks great to the sides of our bathtub....,5,This sticks great to the sides of our bathtub ...,1
61629,67837,"Fisher-Price 2-in-1 Projection Mobile, Preciou...",This thing is awesome! I only wish i had gotte...,5,This thing is awesome I only wish i had gotten...,1
25413,27840,Pigeon Baby Nose Cleaning Tweezers Pigeon (Mad...,"Honestly, this product is not worth the $7.45 ...",1,Honestly this product is not worth the 745 I s...,-1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## 4. Split data into training and test sets
Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [97]:
#train_data, test_data = products.random_split(.8, seed=1)
#print len(train_data)
#print len(test_data)
train_indices = pd.read_json('module-2-assignment-train-idx.json')[0]
test_indices  = pd.read_json('module-2-assignment-test-idx.json')[0]

train_data = products.loc[train_indices]
test_data = products.loc[test_indices]

train_data = train_data.reset_index()
test_data =  test_data.reset_index()
train_data = train_data.drop(['index','level_0'], axis= 1  )
test_data  = test_data.drop (['index','level_0'], axis= 1  )

y_train = train_data['sentiment']
y_test  = test_data['sentiment']


#train_data.head()
#test_data.head()
products.shape, train_data.shape, test_data.shape

((166752, 6), (133416, 5), (33336, 5))

In [98]:
test_data.head()

,name,review,rating,review_clean,sentiment
0,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4,This has been an easy way for my nanny to reco...,1
1,"Baby Tracker&reg; - Daily Childcare Journal, S...",I love this journal and our nanny uses it ever...,4,I love this journal and our nanny uses it ever...,1
2,Nature's Lullabies First Year Sticker Calendar,"I love this little calender, you can keep trac...",5,I love this little calender you can keep track...,1
3,Nature's Lullabies Second Year Sticker Calendar,I had a hard time finding a second year calend...,5,I had a hard time finding a second year calend...,1
4,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,One of babys first and favorite books and it i...,1


## 5. Build the word count vector for each review
Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

```python
def count_words(string):
    string= string.lower().split(' ')
    dt ={}
    for word in string:
        if len(word)>0:
            dt[word] = dt.get(word,0) + 1
    return dt

#count_words( product['review'][1]) 

products['word_count'] = products['review_clean'].apply(count_words)
print (products.loc[269]['word_count'])
products.head()
```


### We will now compute the word count for each word that appears in the reviews. 

* A vector consisting of word counts is often referred to as **bag-of-word features**. 
Since most words occur in only a few reviews, word count vectors are sparse. 
For this reason, scikit-learn and many other tools use **sparse matrices** to store a collection of word count vectors. 
Refer to appropriate manuals to produce sparse word count vectors. 

* General steps for extracting word count vectors are as follows:
  1. Learn a vocabulary (set of all words) from the training data. Only the words that show up in the training data will be considered for feature extraction.
  * Compute the occurrences of the words in each review and collect them into a row vector.
  * Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix train_matrix.
  * Using the same mapping between words and columns, convert the test data into a sparse matrix test_matrix.

The following cell uses CountVectorizer in scikit-learn. Notice the token_pattern argument in the constructor.
* Keep in mind that the test data must be transformed in the same way as the training data.

In [99]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')  
# \w: equal to [a-zA-Z0-9_]
# + : >=1 repetitions
# \b: Matches the empty string, but only at the beginning or end of a word
# Use this token pattern to keep single-letter words

# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform( train_data['review_clean'])

# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform( test_data['review_clean'])

In [100]:
test_data.head()

,name,review,rating,review_clean,sentiment
0,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4,This has been an easy way for my nanny to reco...,1
1,"Baby Tracker&reg; - Daily Childcare Journal, S...",I love this journal and our nanny uses it ever...,4,I love this journal and our nanny uses it ever...,1
2,Nature's Lullabies First Year Sticker Calendar,"I love this little calender, you can keep trac...",5,I love this little calender you can keep track...,1
3,Nature's Lullabies Second Year Sticker Calendar,I had a hard time finding a second year calend...,5,I had a hard time finding a second year calend...,1
4,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,One of babys first and favorite books and it i...,1


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. 

## 7. Learn a logistic regression classifier using the training data. 
* If you are using scikit-learn, you should create an instance of the LogisticRegression class and then call the method fit() to train the classifier. 
* This model should use the sparse word count matrix (train_matrix) as features and the column sentiment of train_data as the target. 
* Use the default values for other parameters. Call this model sentiment_model.

* There should be over 100,000 coefficients in this sentiment_model. 
  * Recall from the lecture that positive weights w_j correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 
  * Calculate the number of positive (>= 0, which is actually nonnegative) coefficients.

In [101]:
clf = LogisticRegression()
sentiment_model = clf.fit(X = train_matrix, y = y_train)

**Aside**. You may get a warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [102]:
weights = sentiment_model.coef_
weights
#weights.column_names()

array([[ -1.24018043e+00,   2.90676284e-05,   2.75328664e-02, ...,
          1.26018688e-02,   2.70953145e-03,  -4.15519037e-05]])

There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [103]:
num_positive_weights = ( weights> 0).sum()
num_negative_weights = ( weights< 0).sum()

print "Number of positive weights: %s " % num_positive_weights
print "Number of negative weights: %s " % num_negative_weights

Number of positive weights: 86908 
Number of negative weights: 34804 


**Quiz Question:** How many weights are >= 0?

In [104]:
( weights>= 0).sum()

86908

## 8. Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [105]:
sample_test_data = test_data[10:13]
print sample_test_data['rating']
sample_test_data

10    5
11    2
12    1
Name: rating, dtype: int64


,name,review,rating,review_clean,sentiment
10,Our Baby Girl Memory Book,Absolutely love it and all of the Scripture in...,5,Absolutely love it and all of the Scripture in...,1
11,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend The deca...,-1
12,New Style Trailing Cherry Blossom Tree Decal R...,Was so excited to get this product for my baby...,1,Was so excited to get this product for my baby...,-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [107]:
sample_test_data.loc[10,'review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [108]:
sample_test_data.loc[11,'review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores**. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

Use a pre-built function in your tool to calculate the score of each data point in sample_test_data. In scikit-learn, you can call the decision_function() function.

Hint: You'd probably need to convert sample_test_data into the sparse matrix format first.
### 8.1 Use decision_function from LogisticRegression model to get scores

In [109]:
# onvert sample_test_data into the sparse matrix format
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])

scores = sentiment_model.decision_function(sample_test_matrix)
print(scores)

[  5.60155603  -3.17063642 -10.42378904]


### 8.2 Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [110]:
y_hat =map(lambda x:  +1 if x >0 else -1, scores )
y_hat

[1, -1, -1]

Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from sklearn.

In [111]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_matrix)

Class predictions according to GraphLab Create:
[ 1 -1 -1]


**Checkpoint**: Make sure your class predictions match with the one obtained from sklearn.

### 8.3 Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [112]:
def sigmoid(scores):
    return [1/(1+ np.exp(- score)) for score in scores ] 
prob = sigmoid(scores)


assert prob == sentiment_model.predict_proba(sample_test_matrix)[:,1].tolist()

**Checkpoint**: Make sure your probability predictions match the ones obtained from sklearn.

In [113]:
print "Class predictions according to sklearn:" 
print  sentiment_model.predict_proba(sample_test_matrix)[:,1]

Class predictions according to sklearn:
[  9.96321467e-01   4.02858024e-02   2.97161836e-05]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# 9. Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_matrix**, and use sklearn to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [114]:
test_scores = sentiment_model.decision_function(test_matrix)
test_prob = sentiment_model.predict_proba(test_matrix)[:,1]

In [132]:
# Top 20 positive reviews 
test_data.loc[ np.argsort(test_prob)[-20:],'name' ]

26838    Baby Jogger City Mini GT Double Stroller, Shad...
30076    Ikea 36 Pcs Kalas Kids Plastic BPA Free Flatwa...
33060    Summer Infant Wide View Digital Color Video Mo...
26830    Baby Jogger City Mini GT Single Stroller, Shad...
24899           Graco Pack 'n Play Element Playard - Flint
15732      Baby Einstein Around The World Discovery Center
9125            P'Kolino Silly Soft Seating in Tias, Green
17558    Freemie Hands-Free Concealable Breast Pump Col...
20743    Fisher-Price Cradle 'N Swing,  My Little Snuga...
30634    Graco FastAction Fold Jogger Click Connect Str...
11923         Evenflo 6 Pack Classic Glass Bottle, 4-Ounce
4140        Britax Decathlon Convertible Car Seat, Tiffany
30535    Buttons Cloth Diaper Cover - One Size - 8 Colo...
18112    Infantino Wrap and Tie Baby Carrier, Black Blu...
9555     Evenflo X Sport Plus Convenience Stroller - Ch...
14482    Simple Wishes Hands-Free Breastpump Bra, Pink,...
24286                    Britax 2012 B-Agile Stroller, R

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [136]:
test_data.loc[ np.argsort(test_prob)[:20],'name' ]

2931           Fisher-Price Ocean Wonders Aquarium Bouncer
21700    Levana Safe N'See Digital Video Baby Monitor w...
13939       Safety 1st Exchangeable Tip 3 in 1 Thermometer
8818     Adiri BPA Free Natural Nurser Ultimate Bottle ...
28184    VTech Communications Safe &amp; Sounds Full Co...
17069    The First Years True Choice P400 Premium Digit...
9655                   Safety 1st High-Def Digital Monitor
14711                Cloth Diaper Sprayer--styles may vary
1942                     Philips AVENT Newborn Starter Set
20594    Motorola Digital Video Baby Monitor with Room ...
10814               Ellaroo Mei Tai Baby Carrier - Hershey
1810          Cosco Alpha Omega Elite Convertible Car Seat
7310     Chicco Cortina KeyFit 30 Travel System in Adve...
31226    Belkin WeMo Wi-Fi Baby Monitor for Apple iPhon...
13751           Peg-Perego Tatamia High Chair, White Latte
27231                     NUK Cook-n-Blend Baby Food Maker
28120    VTech Communications Safe &amp; Sound Digital .

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## 10. Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [125]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    pred = model.predict(data)
    
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    num_correct = sum( pred == true_labels)

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy = num_correct/ len(true_labels)
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [129]:
acc = get_classification_accuracy(sentiment_model, test_matrix, test_data['sentiment'])
print('Accuracy is ' + "{:.2f}".format(acc))

Accuracy is 0.93


**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## 11. Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [130]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [131]:
len(significant_words)

20

Compute a new set of word count vectors using only these words. The CountVectorizer class has a parameter that lets you limit the choice of words when building word count vectors.


In [134]:
vectorizer_word_subset = CountVectorizer( vocabulary = significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform( train_data['review_clean'] )
test_matrix_word_subset  = vectorizer_word_subset.transform( test_data['review_clean'] )


Compute word count vectors for the training and test data and obtain the sparse matrices train_matrix_word_subset and test_matrix_word_subset, respectively.

## 12. Train a logistic regression model on a subset of data

Now build a logistic regression classifier with **train_matrix_word_subset** as features and **sentiment** as the target. Call this model simple_model.

In [145]:
simple_model = LogisticRegression()
simple_model.fit( train_matrix_word_subset, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [146]:
get_classification_accuracy(simple_model, test_matrix_word_subset , y_test)

0.86936045116390692

Now, we will inspect the weights (coefficients) of the **simple_model**:
* First, build a table to store (word, coefficient) pairs. If you are using SFrame with scikit-learn, you can combine words with coefficients by running


In [147]:
simple_model.coef_

array([[ 1.36368976,  0.94399959,  1.19253827,  0.08551278,  0.52018576,
         1.50981248,  1.67307389,  0.50376046,  0.19090857,  0.05885467,
        -1.65157634, -0.20956286, -0.51137963, -2.03369861, -2.34829822,
        -0.62116877, -0.32055624, -0.89803074, -0.36216674, -2.10933109]])

* Sort the data frame by the coefficient value in descending order to obtain the coefficients with the most positive effect on the sentiment.

In [160]:
coef = pd.Series(simple_model.coef_[0].tolist(), significant_words)
coef.sort_values(ascending= False)

loves           1.673074
perfect         1.509812
love            1.363690
easy            1.192538
great           0.944000
little          0.520186
well            0.503760
able            0.190909
old             0.085513
car             0.058855
less           -0.209563
product        -0.320556
would          -0.362167
even           -0.511380
work           -0.621169
money          -0.898031
broke          -1.651576
waste          -2.033699
return         -2.109331
disappointed   -2.348298
dtype: float64

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [161]:
sum(coef > 0 )

10

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [181]:
# get indice of significant_words in vectorizer :
word_index = [vectorizer.vocabulary_[word]for word in significant_words] 

# find out the corresponding coef in sentiment_model
sent_coef = pd.Series( sentiment_model.coef_[0][word_index], index = significant_words)
sent_coef.sort_values( ascending= False)

perfect         1.865770
love            1.582778
loves           1.522427
easy            1.361831
great           1.233162
little          0.640738
well            0.542981
able            0.391330
car             0.123763
old             0.054125
product        -0.191745
less           -0.276866
would          -0.286739
work           -0.463224
even           -0.464966
money          -0.786713
broke          -1.396763
return         -1.662928
waste          -1.997541
disappointed   -2.196958
dtype: float64

# 13. Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [173]:
get_classification_accuracy(sentiment_model, train_matrix , y_train)

0.96848204113449665

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [174]:
get_classification_accuracy(simple_model, train_matrix_word_subset , y_train)

0.8668225700065959

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [175]:
get_classification_accuracy(sentiment_model, test_matrix , y_test)

0.93229541636669067

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [176]:
get_classification_accuracy(simple_model, test_matrix_word_subset , y_test)

0.86936045116390692

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [177]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

112164
21252


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [179]:
print("{:.2f}".format(sum(y_test==1 )/len(y_test)) )

0.84


**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?